In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchtsmixer import TSMixer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from timeview.basis import BSplineBasis
from timeview.knot_selection import calculate_knot_placement
from src.training import train_tsmixer,train_latent_tsmixer,run_trials
from AR_DS_experiments import execute_trials
#add a path to the system path
import sys
sys.path.append('../')

from experiments.datasets import *


seed=42
torch.manual_seed(seed)

# AutoRegresive experiments

## Synthetic

In [2]:
dataset_dict={'n_samples':200,'n_timesteps':60}
trial_dict={'dataset_name':"SineTransDataset_AR",'dataset_params':dataset_dict,'sequence_length':40,'prediction_length':20,'input_channels':1,'output_channels':1,'n_samples':200,'n_timesteps':60, 'B':5,'epochs':200,'n_trials':10}

execute_trials(**trial_dict)

(200, 60)
torch.Size([200, 40, 1]) torch.Size([200, 20, 1])


100%|██████████| 10/10 [00:25<00:00,  2.51s/it]
c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Mean Test Loss: 0.0032, Std Test Loss: 0.0048
(2.0338983050847457, 3.0)


100%|██████████| 10/10 [00:45<00:00,  4.60s/it]

Mean Test Loss: 0.0034, Std Test Loss: 0.0050


In [2]:
sequence_length = 40
prediction_length = 20
input_channels = 1
output_channels = 1
n_samples=200
n_timesteps=sequence_length+prediction_length

#generate a Sine dataset
#Dataset_generator=SineTransDataset_AR(n_samples=n_samples, n_timesteps=n_timesteps)
dataset_name="SineTransDataset_AR"
Dataset_generator=eval(dataset_name)(n_samples=n_samples, n_timesteps=n_timesteps)
X,ts,ys=Dataset_generator.get_X_ts_ys()

Ys=np.array(ys)
X_torch=torch.tensor(Ys[:,:sequence_length]).float().reshape(n_samples,sequence_length,1)
Y_torch=torch.tensor(Ys[:,sequence_length:]).float().reshape(n_samples,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)


B=5
ts_N=[arr[sequence_length:] for arr in ts]
ys_N=[arr[sequence_length:] for arr in ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=ts_N[0][-1], Ti=ts_N[0][0] ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(ts_N[0][0],ts_N[0][-1]),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(ts)[:,sequence_length:]))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)



mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([200, 40, 1]) torch.Size([200, 20, 1])


100%|██████████| 10/10 [00:24<00:00,  2.40s/it]
c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Mean Test Loss: 0.0032, Std Test Loss: 0.0048


100%|██████████| 10/10 [01:05<00:00,  6.54s/it]

Mean Test Loss: 0.0034, Std Test Loss: 0.0050


In [3]:
sequence_length = 40
prediction_length = 20
input_channels = 1
output_channels = 1
n_samples=900
n_timesteps=sequence_length+prediction_length

#generate a Sine dataset
Dataset_generator=BetaDataset(n_samples=n_samples, n_timesteps=n_timesteps)
X,ts,ys=Dataset_generator.get_X_ts_ys()

Ys=np.array(ys)
X_torch=torch.tensor(Ys[:,:sequence_length]).float().reshape(n_samples,sequence_length,1)
Y_torch=torch.tensor(Ys[:,sequence_length:]).float().reshape(n_samples,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

B=5
ts_N=[arr[sequence_length:] for arr in ts]
ys_N=[arr[sequence_length:] for arr in ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=ts_N[0][-1], Ti=ts_N[0][0] ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(ts_N[0][0],ts_N[0][-1]),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(ts)[:,sequence_length:]))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([900, 40, 1]) torch.Size([900, 20, 1])


100%|██████████| 10/10 [03:03<00:00, 18.32s/it]


Mean Test Loss: 0.0075, Std Test Loss: 0.0019


c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [05:38<00:00, 33.88s/it]

Mean Test Loss: 0.0099, Std Test Loss: 0.0018


In [3]:
sequence_length = 40
prediction_length = 20
input_channels = 1
output_channels = 1
n_samples=900


dataset_dict={'n_samples':900,'n_timesteps':60}
trial_dict={'dataset_name':"BetaDataset",'dataset_params':dataset_dict,'sequence_length':40,'prediction_length':20,'input_channels':1,'output_channels':1,'n_samples':900,'n_timesteps':60, 'B':5,'epochs':200,'n_trials':10}

execute_trials(**trial_dict)

torch.Size([900, 40, 1]) torch.Size([900, 20, 1])


100%|██████████| 10/10 [02:32<00:00, 15.20s/it]


Mean Test Loss: 0.0075, Std Test Loss: 0.0019


c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [04:08<00:00, 24.89s/it]

Mean Test Loss: 0.0099, Std Test Loss: 0.0018


In [4]:
sequence_length = 40
prediction_length = 20
input_channels = 1
output_channels = 1
n_samples=2000
n_timesteps=sequence_length+prediction_length


#generate a Tumor dataset
args={
'n_samples': n_samples,'n_time_steps': n_timesteps,'time_horizon': 1.0,'noise_std': 0.0,'seed': 0,'equation': 'wilkerson'
}

Dataset_generator=SyntheticTumorDataset(**args)

X,ts,ys=Dataset_generator.get_X_ts_ys()

Ys=np.array(ys)
X_torch=torch.tensor(Ys[:,:sequence_length]).float().reshape(n_samples,sequence_length,1)
Y_torch=torch.tensor(Ys[:,sequence_length:]).float().reshape(n_samples,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

B=9
ts_N=[arr[sequence_length:] for arr in ts]
ys_N=[arr[sequence_length:] for arr in ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=ts_N[0][-1], Ti=ts_N[0][0] ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(ts_N[0][0],ts_N[0][-1]),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(ts)[:,sequence_length:]))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([2000, 40, 1]) torch.Size([2000, 20, 1])


100%|██████████| 10/10 [06:23<00:00, 38.37s/it]


Mean Test Loss: 0.0041, Std Test Loss: 0.0011


c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [11:57<00:00, 71.72s/it]

Mean Test Loss: 0.0046, Std Test Loss: 0.0024


In [7]:
#generate a Tumor dataset-+
dataset_dict={
'n_samples': 2000,'n_time_steps': 60,'time_horizon': 1.0,'noise_std': 0.0,'seed': 0,'equation': 'wilkerson'
}
trial_dict={'dataset_name':"SyntheticTumorDataset",'dataset_params':dataset_dict,'sequence_length':40,'prediction_length':20,'input_channels':1,'output_channels':1,'n_samples':2000,'n_timesteps':60, 'B':9,'epochs':200,'n_trials':10}

execute_trials(**trial_dict)

torch.Size([2000, 40, 1]) torch.Size([2000, 20, 1])


100%|██████████| 10/10 [05:39<00:00, 33.97s/it]


Mean Test Loss: 0.0035, Std Test Loss: 0.0011


c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [09:57<00:00, 59.77s/it]

Mean Test Loss: 0.0046, Std Test Loss: 0.0024


## Real datasets

In [5]:
os.chdir('../experiments')
Dataset_generator=FLChainDataset()
X,ts,ys=Dataset_generator.get_X_ts_ys()

sequence_length = 10
prediction_length = 10
input_channels = 1
output_channels = 1

Ys=np.array( ys)
Ts=np.array(ts)

X_torch=torch.tensor(Ys[:,:sequence_length]).float().reshape(-1,sequence_length,1)
Y_torch=torch.tensor(Ys[:,sequence_length:]).float().reshape(-1,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

B=9
ts_N=[arr[sequence_length:] for arr in ts]
ys_N=[arr[sequence_length:] for arr in ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=1.0, Ti=0.0 ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(0.0,1.0),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(Ts)[:,sequence_length:]))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([5499, 10, 1]) torch.Size([5499, 10, 1])


100%|██████████| 10/10 [16:01<00:00, 96.13s/it]


Mean Test Loss: 0.0009, Std Test Loss: 0.0001


c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [31:32<00:00, 189.29s/it]

Mean Test Loss: 0.0009, Std Test Loss: 0.0001


In [5]:
os.chdir('../experiments')
dataset_dict={}
trial_dict={'dataset_name':"FLChainDataset",'dataset_params':dataset_dict,'sequence_length':10,'prediction_length':10,'input_channels':1,'output_channels':1,'n_samples':5499,'n_timesteps':20, 'B':9,'epochs':200,'n_trials':10}

execute_trials(**trial_dict)

torch.Size([5499, 10, 1]) torch.Size([5499, 10, 1])


100%|██████████| 10/10 [11:42<00:00, 70.25s/it]


Mean Test Loss: 0.0010, Std Test Loss: 0.0001


c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
100%|██████████| 10/10 [24:19<00:00, 145.94s/it]

Mean Test Loss: 0.0009, Std Test Loss: 0.0001


In [4]:
os.chdir('../experiments')
dataset_dict={  "lot": "all",
                "include_lot_as_feature": True,
                "downsample": True,
                "more_samples": 0,
                "specimen": "all",
                "max_strain": 0.2}

trial_dict={'dataset_name':"StressStrainDataset",'dataset_params':dataset_dict,'sequence_length':140,'prediction_length':70,'input_channels':1,'output_channels':1,'n_samples':47,'n_timesteps':210,
                "fixed_len":210, 'B':9,'epochs':200,'n_trials':10,'Ti':0.0,'T':1.0}

execute_trials(**trial_dict)

(47, 210)
torch.Size([47, 140, 1]) torch.Size([47, 70, 1])


100%|██████████| 10/10 [00:12<00:00,  1.22s/it]
c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Mean Test Loss: 0.0018, Std Test Loss: 0.0025
(0.0640975546777966, 0.119137177682097)


100%|██████████| 10/10 [00:17<00:00,  1.80s/it]

Mean Test Loss: 0.0018, Std Test Loss: 0.0026


In [6]:
Dataset_generator=StressStrainDataset(**{
                "lot": "all",
                "include_lot_as_feature": True,
                "downsample": True,
                "more_samples": 0,
                "specimen": "all",
                "max_strain": 0.2})
X,ts,ys=Dataset_generator.get_X_ts_ys()

sequence_length = 140
prediction_length = 70
input_channels = 1
output_channels = 1
fixed_length=210
Ys=np.array([y[:210] for y in ys if len(y)>=210])
Ts=np.array([t[:210] for t in ts if len(t)>=210])
X=X[X.index.isin([i for i in range(len(ys)) if len(ys[i])>=prediction_length])]

X_torch=torch.tensor(Ys[:,:sequence_length]).float().reshape(-1,sequence_length,1)
Y_torch=torch.tensor(Ys[:,sequence_length:]).float().reshape(-1,prediction_length,1)
print(X_torch.shape,Y_torch.shape)

model= TSMixer(
    sequence_length=sequence_length,
    prediction_length=prediction_length,
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_tsmixer,
    model,
    X_torch,
    Y_torch,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

B=9
ts_N=[arr[sequence_length:] for arr in Ts]
ys_N=[arr[sequence_length:] for arr in Ys]

internal_knots=calculate_knot_placement(ts_N, ys_N, n_internal_knots=B-2, T=1 ,seed=0, verbose=False)

bspline=BSplineBasis(n_basis=B,t_range=(0,1),internal_knots=internal_knots)
Phis = list(bspline.get_all_matrices(np.array(Ts)[:,sequence_length:]))

latent_model = TSMixer(
    sequence_length=sequence_length,   # same as time steps in X
    prediction_length=B, # number of spline coefficients to predict
    input_channels=input_channels,
    output_channels=output_channels,
)


mean_loss,std_loss=run_trials(
    train_latent_tsmixer,
    latent_model,
    X_torch,
    Y_torch,
    Phis=Phis,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    epochs=200,
    batch_size=32,
    lr=1e-2,
    device="cpu",
    n_trials=10,
    seed=42,
)

torch.Size([47, 140, 1]) torch.Size([47, 70, 1])


100%|██████████| 10/10 [00:23<00:00,  2.32s/it]
c:\Users\nicolas.i.fierro\AppData\Local\anaconda3\envs\timeview\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Mean Test Loss: 0.0018, Std Test Loss: 0.0025


100%|██████████| 10/10 [00:17<00:00,  1.72s/it]

Mean Test Loss: 0.0018, Std Test Loss: 0.0026
